In [2]:
from google.colab import files
uploaded = files.upload()


Saving heart_attack_prediction_indonesia.csv to heart_attack_prediction_indonesia.csv


In [3]:
!pip install pandas numpy scikit-learn matplotlib seaborn

In [18]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('heart_attack_prediction_indonesia.csv')
print(df.columns)

Index(['age', 'gender', 'region', 'income_level', 'hypertension', 'diabetes',
       'cholesterol_level', 'obesity', 'waist_circumference', 'family_history',
       'smoking_status', 'alcohol_consumption', 'physical_activity',
       'dietary_habits', 'air_pollution_exposure', 'stress_level',
       'sleep_hours', 'blood_pressure_systolic', 'blood_pressure_diastolic',
       'fasting_blood_sugar', 'cholesterol_hdl', 'cholesterol_ldl',
       'triglycerides', 'EKG_results', 'previous_heart_disease',
       'medication_usage', 'participated_in_free_screening', 'heart_attack'],
      dtype='object')


In [7]:
# Normalisasi nama kolom
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Lihat nama kolom
print(df.columns)

Index(['age', 'gender', 'region', 'income_level', 'hypertension', 'diabetes',
       'cholesterol_level', 'obesity', 'waist_circumference', 'family_history',
       'smoking_status', 'alcohol_consumption', 'physical_activity',
       'dietary_habits', 'air_pollution_exposure', 'stress_level',
       'sleep_hours', 'blood_pressure_systolic', 'blood_pressure_diastolic',
       'fasting_blood_sugar', 'cholesterol_hdl', 'cholesterol_ldl',
       'triglycerides', 'ekg_results', 'previous_heart_disease',
       'medication_usage', 'participated_in_free_screening', 'heart_attack'],
      dtype='object')


In [8]:
#Pilih Fitur & Target

In [9]:
# Pilih fitur-fitur yang relevan
features = [
    'age',
    'hypertension',
    'diabetes',
    'cholesterol_level',
    'obesity',
    'smoking_status',
    'air_pollution_exposure',
    'stress_level',
    'sleep_hours'
]

target = 'heart_attack'

# Filter data lengkap (tanpa NaN)
df_clean = df.dropna(subset=features + [target])

# Pisahkan fitur dan target
X = df_clean[features]
y = df_clean[target]


In [10]:
#Split Data & Scaling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


ValueError: could not convert string to float: 'Never'

In [11]:
print(X.dtypes)

age                         int64
hypertension                int64
diabetes                    int64
cholesterol_level           int64
obesity                     int64
smoking_status             object
air_pollution_exposure     object
stress_level               object
sleep_hours               float64
dtype: object


In [12]:
from sklearn.preprocessing import LabelEncoder

# Salin dataframe
X_encoded = X.copy()

# Label encoding untuk fitur kategorikal
le = LabelEncoder()
X_encoded['smoking_status'] = le.fit_transform(X_encoded['smoking_status'])

# Cek hasil encode
print(X_encoded['smoking_status'].unique())


[1 2 0]


In [13]:
print(X_encoded.dtypes)

age                         int64
hypertension                int64
diabetes                    int64
cholesterol_level           int64
obesity                     int64
smoking_status              int64
air_pollution_exposure     object
stress_level               object
sleep_hours               float64
dtype: object


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split ulang dengan data yang sudah diencode
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

# Scaling fitur numerik
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


ValueError: could not convert string to float: 'High'

In [15]:
from sklearn.preprocessing import LabelEncoder

# Salin dataframe
X_encoded = X.copy()

# Ubah semua kolom bertipe 'object' menjadi numerik
for col in X_encoded.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col])

# Cek hasil encoding
print(X_encoded.dtypes)


age                         int64
hypertension                int64
diabetes                    int64
cholesterol_level           int64
obesity                     int64
smoking_status              int64
air_pollution_exposure      int64
stress_level                int64
sleep_hours               float64
dtype: object


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [17]:
print(type(X_train_scaled), X_train_scaled.shape)


<class 'numpy.ndarray'> (126684, 9)


In [20]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# Prediksi dan threshold
y_pred_lr = lr.predict(X_test_scaled)
y_pred_lr_class = (y_pred_lr >= 0.5).astype(int)

print("Linear Regression")
print("Akurasi:", accuracy_score(y_test, y_pred_lr_class))
print(confusion_matrix(y_test, y_pred_lr_class))
print(classification_report(y_test, y_pred_lr_class))


Linear Regression
Akurasi: 0.6977992485238862
[[15550  3356]
 [ 6215  6550]]
              precision    recall  f1-score   support

           0       0.71      0.82      0.76     18906
           1       0.66      0.51      0.58     12765

    accuracy                           0.70     31671
   macro avg       0.69      0.67      0.67     31671
weighted avg       0.69      0.70      0.69     31671



In [22]:
#Polynomial Regression (degree=2)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

lr_poly = LinearRegression()
lr_poly.fit(X_train_poly, y_train)

y_pred_poly = lr_poly.predict(X_test_poly)
y_pred_poly_class = (y_pred_poly >= 0.5).astype(int)

print("Polynomial Regression (degree=2)")
print("Akurasi:", accuracy_score(y_test, y_pred_poly_class))
print(confusion_matrix(y_test, y_pred_poly_class))
print(classification_report(y_test, y_pred_poly_class))


Polynomial Regression (degree=2)
Akurasi: 0.7038300022102238
[[15529  3377]
 [ 6003  6762]]
              precision    recall  f1-score   support

           0       0.72      0.82      0.77     18906
           1       0.67      0.53      0.59     12765

    accuracy                           0.70     31671
   macro avg       0.69      0.68      0.68     31671
weighted avg       0.70      0.70      0.70     31671



In [23]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)

print("KNN (k=5)")
print("Akurasi:", accuracy_score(y_test, y_pred_knn))
print(confusion_matrix(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))


KNN (k=5)
Akurasi: 0.6632566069906224
[[14240  4666]
 [ 5999  6766]]
              precision    recall  f1-score   support

           0       0.70      0.75      0.73     18906
           1       0.59      0.53      0.56     12765

    accuracy                           0.66     31671
   macro avg       0.65      0.64      0.64     31671
weighted avg       0.66      0.66      0.66     31671



In [24]:
nb = GaussianNB()
nb.fit(X_train_scaled, y_train)
y_pred_nb = nb.predict(X_test_scaled)

print("Naive Bayes")
print("Akurasi:", accuracy_score(y_test, y_pred_nb))
print(confusion_matrix(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

Naive Bayes
Akurasi: 0.6939471440750213
[[14718  4188]
 [ 5505  7260]]
              precision    recall  f1-score   support

           0       0.73      0.78      0.75     18906
           1       0.63      0.57      0.60     12765

    accuracy                           0.69     31671
   macro avg       0.68      0.67      0.68     31671
weighted avg       0.69      0.69      0.69     31671



In [25]:
print("Summary Akurasi:")
print("Linear Regression     :", accuracy_score(y_test, y_pred_lr_class))
print("Polynomial Regression :", accuracy_score(y_test, y_pred_poly_class))
print("KNN                   :", accuracy_score(y_test, y_pred_knn))
print("Naive Bayes           :", accuracy_score(y_test, y_pred_nb))


Summary Akurasi:
Linear Regression     : 0.6977992485238862
Polynomial Regression : 0.7038300022102238
KNN                   : 0.6632566069906224
Naive Bayes           : 0.6939471440750213
